In [1]:
import pandas as pd
import numpy as np

In [2]:
throws = pd.read_csv("throws_clean.csv")

throws.head()

,ThrowID,X2,GameID,throw_team_score,def_team_score,Score,point_per_game,poss_per_point,score_on_poss,Direction,...,GEO_Yend,RO_Xstart,RO_Ystart,RO_Xend,RO_Yend,Xdist,Ydist,Dist,pull,goal
0,1-Jan,2,1,0,0,0-0,1,1,1,1,...,-10.5,-32.5,-10.5,-24.5,-10.5,8.0,0.0,8.0,NaN,0
1,2-Jan,3,1,0,0,0-0,1,1,1,1,...,5.5,-24.5,-10.5,-25.5,5.5,-1.0,16.0,16.0,NaN,0
2,3-Jan,4,1,0,0,0-0,1,1,1,1,...,14.5,-25.5,5.5,-18.5,14.5,7.0,9.0,11.4,NaN,0
3,4-Jan,5,1,0,0,0-0,1,1,1,1,...,18.5,-18.5,14.5,-4.5,18.5,14.0,4.0,14.6,NaN,0
4,5-Jan,6,1,0,0,0-0,1,1,1,1,...,14.5,-4.5,18.5,-16.5,14.5,-12.0,-4.0,12.6,NaN,0
